# Import libraries

In [1]:
# ===== BASIC LIBRARIES =====
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()


# ===== MODELS =====
# simpler models
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

# ensemmble methods
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier, StackingClassifier
import xgboost as xgb; xgb.set_config(verbosity=0) # to deactivate warning about change in library
from xgboost import XGBClassifier


# ===== HYPERPARAMETER TUNING =====
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate


# ===== PERFORMANCE MEASURES =====
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score, roc_auc_score


# ===== EXPORTS MODELS =====
import pickle


pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:.3f}'.format

# Load preprocessed data

In [2]:
train_set = pd.read_csv('train_churndata_preprocessed.csv')
train_set.head()

Churn  Tenure  CityTier  WarehouseToHome  HourSpendOnApp  \
0      0  -0.667     0.000           -0.193           0.000   
1      0   0.667     0.000           -0.093           1.000   
2      0  -0.750     0.000           -0.553          -0.066   
3      0   0.100     0.000           -0.701          -1.000   
4      0  -0.417     0.000           -0.302          -1.000   

   NumberOfDeviceRegistered  SatisfactionScore  NumberOfAddress  Complain  \
0                     0.000              0.500            0.000     1.000   
1                     1.000              0.000            0.000     1.000   
2                     0.000              0.000           -0.250     0.000   
3                    -1.000             -0.500           -0.250     1.000   
4                    -1.000             -0.500           -0.500     1.000   

   OrderAmountHikeFromlastYear  CouponUsed  OrderCount  DaySinceLastOrder  \
0                       -0.400       0.000       0.000              0.000   
1                        0.000       1.000       0.000              1.200   
2                       -0.600      -1.000      -0.493             -0.200   
3                        1.600       0.000      -0.493             -0.600   
4                       -0.600      -1.000      -0.493              0.000   

   CashbackAmount  PreferredLoginDevice_Mobile Phone  \
0          -0.277                              1.000   
1           0.043                              1.000   
2          -0.665                              0.000   
3          -0.931                              0.000   
4          -0.308                              0.000   

   PreferredLoginDevice_Phone  PreferredLoginDevice_Computer  \
0                       0.000                          0.000   
1                       0.000                          0.000   
2                       1.000                          0.000   
3                       0.000                          1.000   
4                       1.000                          0.000   

   PreferredPaymentMode_Debit Card  PreferredPaymentMode_Credit Card  \
0                            1.000                             0.000   
1                            0.000                             1.000   
2                            1.000                             0.000   
3                            1.000                             0.000   
4                            0.000                             0.000   

   PreferredPaymentMode_COD  PreferredPaymentMode_UPI  \
0                     0.000                     0.000   
1                     0.000                     0.000   
2                     0.000                     0.000   
3                     0.000                     0.000   
4                     1.000                     0.000   

   PreferredPaymentMode_Cash on Delivery  PreferredPaymentMode_E wallet  \
0                                  0.000                          0.000   
1                                  0.000                          0.000   
2                                  0.000                          0.000   
3                                  0.000                          0.000   
4                                  0.000                          0.000   

   PreferredPaymentMode_CC  Gender_Female  PreferedOrderCat_Mobile Phone  \
0                    0.000          1.000                          1.000   
1                    0.000          1.000                          0.000   
2                    0.000          0.000                          0.000   
3                    0.000          0.000                          1.000   
4                    0.000          0.000                          1.000   

   PreferedOrderCat_Laptop & Accessory  PreferedOrderCat_Mobile  \
0                                0.000                    0.000   
1                                1.000                    0.000   
2                                0.000                    1.000   
3                       

In [3]:
X_train = train_set.drop(['Churn'], axis = 1).values
y_train = train_set['Churn'].values

# Setting things up

In [4]:
metrics = ['accuracy','f1','precision','recall']
models = []
scores = []

# Exploring Models

In [5]:
%%time
exp_models = [LogisticRegression(), 
               KNeighborsClassifier(), 
               SVC(random_state = 42),
               SGDClassifier(random_state = 42),
               RandomForestClassifier(random_state = 42),
               ExtraTreesClassifier(random_state = 42),
               AdaBoostClassifier(random_state = 42),
               GradientBoostingClassifier(random_state = 42),
               XGBClassifier(random_state = 42, use_label_encoder=False)]


for model in exp_models:
    model.fit(X_train, y_train)
    
    cross_val = cross_validate(model, X_train, y_train, cv= 10, scoring = metrics)

    models.append(f'{model.__class__.__name__}')
    scores.append(list(cross_val.values())[2:])

Wall time: 58.6 s


In [6]:
data = []
for i in range(len(models)):
    for j in range(len(metrics)):
        data.append(scores[i][j].mean())

data = np.reshape(data, (len(models), len(metrics)))
        
df_models = pd.DataFrame(data, index = models, columns = metrics)
df_models.style.highlight_max(color = 'lightgreen', axis = 0)

In [7]:
df_models[df_models.f1 > 0.9]

accuracy    f1  precision  recall
KNeighborsClassifier           0.908 0.916      0.847   0.997
SVC                            0.926 0.928      0.901   0.956
RandomForestClassifier         0.980 0.980      0.981   0.979
ExtraTreesClassifier           0.990 0.990      0.988   0.993
GradientBoostingClassifier     0.924 0.920      0.934   0.913
XGBClassifier                  0.976 0.975      0.984   0.968

# Fine Tune models

In [8]:
fine_tuned_models = []
new_models = []
new_scores = []

### KNeighbors Classifier

In [9]:
%%time

'''KNeighborsClassifier
parameters: 
    n_neighbors: number of neighbors to use for kneighbors queries; (default = 5)
                    a smaller number can give the model more flexibility, thus lower bias but higher variance (risk overfitting)
                    Samewise, a higher number can increase bias but lower variance (risk underfitting)
    weights: weight function used in prediction; (default = 'uniform')
    
    leaf_size: leaf size passed to BallTree or KDTree (algorithm); (default = 30)
    '''

param_grid = {'n_neighbors':np.arange(2, 15, 1),
              'weights':['uniform', 'distance'],
              'leaf_size':np.arange(15, 40, 2),
             }

knn_clf = GridSearchCV(estimator = KNeighborsClassifier(n_jobs = -1),
                      param_grid = param_grid, 
                      cv = 10, 
                      scoring = metrics,
                      refit = 'recall',
                      error_score = 'raise', 
                      verbose = 2)
                     
knn_clf.fit(X_train, y_train)

Fitting 10 folds for each of 338 candidates, totalling 3380 fits
[CV] END .......leaf_size=15, n_neighbors=2, weights=uniform; total time=   0.1s
[CV] END .......leaf_size=15, n_neighbors=2, weights=uniform; total time=   0.0s
[CV] END .......leaf_size=15, n_neighbors=2, weights=uniform; total time=   0.0s
[CV] END .......leaf_size=15, n_neighbors=2, weights=uniform; total time=   0.0s
[CV] END .......leaf_size=15, n_neighbors=2, weights=uniform; total time=   0.0s
[CV] END .......leaf_size=15, n_neighbors=2, weights=uniform; total time=   0.0s
[CV] END .......leaf_size=15, n_neighbors=2, weights=uniform; total time=   0.0s
[CV] END .......leaf_size=15, n_neighbors=2, weights=uniform; total time=   0.0s
[CV] END .......leaf_size=15, n_neighbors=2, weights=uniform; total time=   0.0s
[CV] END .......leaf_size=15, n_neighbors=2, weights=uniform; total time=   0.0s
[CV] END ......leaf_size=15, n_neighbors=2, weights=distance; total time=   0.0s
[CV] END ......leaf_size=15, n_neighbors=2, 

[CV] END .......leaf_size=15, n_neighbors=7, weights=uniform; total time=   0.0s
[CV] END .......leaf_size=15, n_neighbors=7, weights=uniform; total time=   0.0s
[CV] END .......leaf_size=15, n_neighbors=7, weights=uniform; total time=   0.0s
[CV] END .......leaf_size=15, n_neighbors=7, weights=uniform; total time=   0.0s
[CV] END .......leaf_size=15, n_neighbors=7, weights=uniform; total time=   0.0s
[CV] END .......leaf_size=15, n_neighbors=7, weights=uniform; total time=   0.0s
[CV] END .......leaf_size=15, n_neighbors=7, weights=uniform; total time=   0.0s
[CV] END .......leaf_size=15, n_neighbors=7, weights=uniform; total time=   0.0s
[CV] END ......leaf_size=15, n_neighbors=7, weights=distance; total time=   0.0s
[CV] END ......leaf_size=15, n_neighbors=7, weights=distance; total time=   0.0s
[CV] END ......leaf_size=15, n_neighbors=7, weights=distance; total time=   0.0s
[CV] END ......leaf_size=15, n_neighbors=7, weights=distance; total time=   0.0s
[CV] END ......leaf_size=15,

[CV] END ......leaf_size=15, n_neighbors=12, weights=uniform; total time=   0.1s
[CV] END ......leaf_size=15, n_neighbors=12, weights=uniform; total time=   0.1s
[CV] END ......leaf_size=15, n_neighbors=12, weights=uniform; total time=   0.1s
[CV] END ......leaf_size=15, n_neighbors=12, weights=uniform; total time=   0.1s
[CV] END ......leaf_size=15, n_neighbors=12, weights=uniform; total time=   0.0s
[CV] END ......leaf_size=15, n_neighbors=12, weights=uniform; total time=   0.1s
[CV] END .....leaf_size=15, n_neighbors=12, weights=distance; total time=   0.0s
[CV] END .....leaf_size=15, n_neighbors=12, weights=distance; total time=   0.0s
[CV] END .....leaf_size=15, n_neighbors=12, weights=distance; total time=   0.0s
[CV] END .....leaf_size=15, n_neighbors=12, weights=distance; total time=   0.0s
[CV] END .....leaf_size=15, n_neighbors=12, weights=distance; total time=   0.0s
[CV] END .....leaf_size=15, n_neighbors=12, weights=distance; total time=   0.0s
[CV] END .....leaf_size=15, 

[CV] END .......leaf_size=17, n_neighbors=4, weights=uniform; total time=   0.0s
[CV] END .......leaf_size=17, n_neighbors=4, weights=uniform; total time=   0.0s
[CV] END .......leaf_size=17, n_neighbors=4, weights=uniform; total time=   0.0s
[CV] END ......leaf_size=17, n_neighbors=4, weights=distance; total time=   0.0s
[CV] END ......leaf_size=17, n_neighbors=4, weights=distance; total time=   0.0s
[CV] END ......leaf_size=17, n_neighbors=4, weights=distance; total time=   0.0s
[CV] END ......leaf_size=17, n_neighbors=4, weights=distance; total time=   0.0s
[CV] END ......leaf_size=17, n_neighbors=4, weights=distance; total time=   0.0s
[CV] END ......leaf_size=17, n_neighbors=4, weights=distance; total time=   0.0s
[CV] END ......leaf_size=17, n_neighbors=4, weights=distance; total time=   0.0s
[CV] END ......leaf_size=17, n_neighbors=4, weights=distance; total time=   0.0s
[CV] END ......leaf_size=17, n_neighbors=4, weights=distance; total time=   0.0s
[CV] END ......leaf_size=17,

[CV] END .......leaf_size=17, n_neighbors=9, weights=uniform; total time=   0.1s
[CV] END ......leaf_size=17, n_neighbors=9, weights=distance; total time=   0.0s
[CV] END ......leaf_size=17, n_neighbors=9, weights=distance; total time=   0.0s
[CV] END ......leaf_size=17, n_neighbors=9, weights=distance; total time=   0.0s
[CV] END ......leaf_size=17, n_neighbors=9, weights=distance; total time=   0.0s
[CV] END ......leaf_size=17, n_neighbors=9, weights=distance; total time=   0.0s
[CV] END ......leaf_size=17, n_neighbors=9, weights=distance; total time=   0.0s
[CV] END ......leaf_size=17, n_neighbors=9, weights=distance; total time=   0.0s
[CV] END ......leaf_size=17, n_neighbors=9, weights=distance; total time=   0.0s
[CV] END ......leaf_size=17, n_neighbors=9, weights=distance; total time=   0.0s
[CV] END ......leaf_size=17, n_neighbors=9, weights=distance; total time=   0.0s
[CV] END ......leaf_size=17, n_neighbors=10, weights=uniform; total time=   0.0s
[CV] END ......leaf_size=17,

...

[CV] END ......leaf_size=37, n_neighbors=12, weights=uniform; total time=   0.0s
[CV] END ......leaf_size=37, n_neighbors=12, weights=uniform; total time=   0.1s
[CV] END ......leaf_size=37, n_neighbors=12, weights=uniform; total time=   0.0s
[CV] END ......leaf_size=37, n_neighbors=12, weights=uniform; total time=   0.1s
[CV] END .....leaf_size=37, n_neighbors=12, weights=distance; total time=   0.0s
[CV] END .....leaf_size=37, n_neighbors=12, weights=distance; total time=   0.0s
[CV] END .....leaf_size=37, n_neighbors=12, weights=distance; total time=   0.0s
[CV] END .....leaf_size=37, n_neighbors=12, weights=distance; total time=   0.0s
[CV] END .....leaf_size=37, n_neighbors=12, weights=distance; total time=   0.0s
[CV] END .....leaf_size=37, n_neighbors=12, weights=distance; total time=   0.0s
[CV] END .....leaf_size=37, n_neighbors=12, weights=distance; total time=   0.0s
[CV] END .....leaf_size=37, n_neighbors=12, weights=distance; total time=   0.0s
[CV] END .....leaf_size=37, 

[CV] END .......leaf_size=39, n_neighbors=4, weights=uniform; total time=   0.0s
[CV] END .......leaf_size=39, n_neighbors=4, weights=uniform; total time=   0.1s
[CV] END ......leaf_size=39, n_neighbors=4, weights=distance; total time=   0.0s
[CV] END ......leaf_size=39, n_neighbors=4, weights=distance; total time=   0.0s
[CV] END ......leaf_size=39, n_neighbors=4, weights=distance; total time=   0.0s
[CV] END ......leaf_size=39, n_neighbors=4, weights=distance; total time=   0.0s
[CV] END ......leaf_size=39, n_neighbors=4, weights=distance; total time=   0.0s
[CV] END ......leaf_size=39, n_neighbors=4, weights=distance; total time=   0.0s
[CV] END ......leaf_size=39, n_neighbors=4, weights=distance; total time=   0.0s
[CV] END ......leaf_size=39, n_neighbors=4, weights=distance; total time=   0.0s
[CV] END ......leaf_size=39, n_neighbors=4, weights=distance; total time=   0.0s
[CV] END ......leaf_size=39, n_neighbors=4, weights=distance; total time=   0.0s
[CV] END .......leaf_size=39

[CV] END ......leaf_size=39, n_neighbors=9, weights=distance; total time=   0.0s
[CV] END ......leaf_size=39, n_neighbors=9, weights=distance; total time=   0.0s
[CV] END ......leaf_size=39, n_neighbors=9, weights=distance; total time=   0.0s
[CV] END ......leaf_size=39, n_neighbors=9, weights=distance; total time=   0.0s
[CV] END ......leaf_size=39, n_neighbors=9, weights=distance; total time=   0.0s
[CV] END ......leaf_size=39, n_neighbors=9, weights=distance; total time=   0.0s
[CV] END ......leaf_size=39, n_neighbors=9, weights=distance; total time=   0.0s
[CV] END ......leaf_size=39, n_neighbors=9, weights=distance; total time=   0.0s
[CV] END ......leaf_size=39, n_neighbors=9, weights=distance; total time=   0.0s
[CV] END ......leaf_size=39, n_neighbors=9, weights=distance; total time=   0.0s
[CV] END ......leaf_size=39, n_neighbors=10, weights=uniform; total time=   0.1s
[CV] END ......leaf_size=39, n_neighbors=10, weights=uniform; total time=   0.0s
[CV] END ......leaf_size=39,

[CV] END .....leaf_size=39, n_neighbors=14, weights=distance; total time=   0.0s
[CV] END .....leaf_size=39, n_neighbors=14, weights=distance; total time=   0.0s
[CV] END .....leaf_size=39, n_neighbors=14, weights=distance; total time=   0.0s
[CV] END .....leaf_size=39, n_neighbors=14, weights=distance; total time=   0.1s
[CV] END .....leaf_size=39, n_neighbors=14, weights=distance; total time=   0.0s
[CV] END .....leaf_size=39, n_neighbors=14, weights=distance; total time=   0.0s
[CV] END .....leaf_size=39, n_neighbors=14, weights=distance; total time=   0.0s
[CV] END .....leaf_size=39, n_neighbors=14, weights=distance; total time=   0.0s
Wall time: 7min 52s


GridSearchCV(cv=10, error_score='raise',
             estimator=KNeighborsClassifier(n_jobs=-1),
             param_grid={'leaf_size': array([15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 35, 37, 39]),
                         'n_neighbors': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14]),
                         'weights': ['uniform', 'distance']},
             refit='recall', scoring=['accuracy', 'f1', 'precision', 'recall'],
             verbose=2)

In [10]:
fine_tuned_models.append(knn_clf.best_estimator_)
knn_clf.best_params_

{'leaf_size': 15, 'n_neighbors': 2, 'weights': 'distance'}

### Support Vector Classification (SVC)

In [11]:
%%time

'''SVC -SupportVectorClassification
parameters: 
    C: regularization parameter; inversely proportional to C; (default = 1.0)
                    
    kernel: kernel type to be used in the algorithm (linear, poly, rbf, sigmoid, precoumpted); (default = 'rbf')
    
    degree: degree of polynomial kernel function; (default = 3)
    
    gamma: kernel coefficient for 'rbf', 'poly' and 'sigmoid'; (default = 'scale' = 1/(n_features * X.var()))
            If set too large, risk of overfitting
    
    tol: tolerance for stopping criterion; (default = 0.001)
    '''

param_grid = {'C':[1.0, 1.5, 2.0, 2.5],
              'kernel':['poly', 'rbf', 'sigmoid'],
              'degree':[2, 3],
              'gamma':['scale', 'auto'],
              'tol':[0.001, 0.003]
             }

svc_clf = GridSearchCV(estimator = SVC(probability = True, class_weight = 'balanced'),
                      param_grid = param_grid, 
                      cv = 5, 
                      scoring = metrics,
                      refit = 'recall',
                      error_score = 'raise', 
                      verbose = 2)
                     
svc_clf.fit(X_train, y_train)

Fitting 5 folds for each of 96 candidates, totalling 480 fits
[CV] END C=1.0, degree=2, gamma=scale, kernel=poly, tol=0.001; total time=   5.5s
[CV] END C=1.0, degree=2, gamma=scale, kernel=poly, tol=0.001; total time=   5.4s
[CV] END C=1.0, degree=2, gamma=scale, kernel=poly, tol=0.001; total time=   4.9s
[CV] END C=1.0, degree=2, gamma=scale, kernel=poly, tol=0.001; total time=   4.8s
[CV] END C=1.0, degree=2, gamma=scale, kernel=poly, tol=0.001; total time=   4.9s
[CV] END C=1.0, degree=2, gamma=scale, kernel=poly, tol=0.003; total time=   4.7s
[CV] END C=1.0, degree=2, gamma=scale, kernel=poly, tol=0.003; total time=   4.3s
[CV] END C=1.0, degree=2, gamma=scale, kernel=poly, tol=0.003; total time=   3.9s
[CV] END C=1.0, degree=2, gamma=scale, kernel=poly, tol=0.003; total time=   4.0s
[CV] END C=1.0, degree=2, gamma=scale, kernel=poly, tol=0.003; total time=   4.0s
[CV] END C=1.0, degree=2, gamma=scale, kernel=rbf, tol=0.001; total time=   5.4s
[CV] END C=1.0, degree=2, gamma=scale

[CV] END C=1.0, degree=3, gamma=auto, kernel=poly, tol=0.003; total time=   5.5s
[CV] END .C=1.0, degree=3, gamma=auto, kernel=rbf, tol=0.001; total time=   5.7s
[CV] END .C=1.0, degree=3, gamma=auto, kernel=rbf, tol=0.001; total time=   6.1s
[CV] END .C=1.0, degree=3, gamma=auto, kernel=rbf, tol=0.001; total time=   5.7s
[CV] END .C=1.0, degree=3, gamma=auto, kernel=rbf, tol=0.001; total time=   5.8s
[CV] END .C=1.0, degree=3, gamma=auto, kernel=rbf, tol=0.001; total time=   5.7s
[CV] END .C=1.0, degree=3, gamma=auto, kernel=rbf, tol=0.003; total time=   5.7s
[CV] END .C=1.0, degree=3, gamma=auto, kernel=rbf, tol=0.003; total time=   6.0s
[CV] END .C=1.0, degree=3, gamma=auto, kernel=rbf, tol=0.003; total time=   5.8s
[CV] END .C=1.0, degree=3, gamma=auto, kernel=rbf, tol=0.003; total time=   6.1s
[CV] END .C=1.0, degree=3, gamma=auto, kernel=rbf, tol=0.003; total time=   6.1s
[CV] END C=1.0, degree=3, gamma=auto, kernel=sigmoid, tol=0.001; total time=   3.2s
[CV] END C=1.0, degree=3,

[CV] END C=1.5, degree=3, gamma=scale, kernel=rbf, tol=0.003; total time=   5.0s
[CV] END C=1.5, degree=3, gamma=scale, kernel=sigmoid, tol=0.001; total time=   3.2s
[CV] END C=1.5, degree=3, gamma=scale, kernel=sigmoid, tol=0.001; total time=   3.4s
[CV] END C=1.5, degree=3, gamma=scale, kernel=sigmoid, tol=0.001; total time=   3.3s
[CV] END C=1.5, degree=3, gamma=scale, kernel=sigmoid, tol=0.001; total time=   3.2s
[CV] END C=1.5, degree=3, gamma=scale, kernel=sigmoid, tol=0.001; total time=   3.3s
[CV] END C=1.5, degree=3, gamma=scale, kernel=sigmoid, tol=0.003; total time=   3.2s
[CV] END C=1.5, degree=3, gamma=scale, kernel=sigmoid, tol=0.003; total time=   3.4s
[CV] END C=1.5, degree=3, gamma=scale, kernel=sigmoid, tol=0.003; total time=   3.3s
[CV] END C=1.5, degree=3, gamma=scale, kernel=sigmoid, tol=0.003; total time=   3.2s
[CV] END C=1.5, degree=3, gamma=scale, kernel=sigmoid, tol=0.003; total time=   3.2s
[CV] END C=1.5, degree=3, gamma=auto, kernel=poly, tol=0.001; total t

[CV] END C=2.0, degree=2, gamma=auto, kernel=sigmoid, tol=0.003; total time=   3.6s
[CV] END C=2.0, degree=3, gamma=scale, kernel=poly, tol=0.001; total time=   4.7s
[CV] END C=2.0, degree=3, gamma=scale, kernel=poly, tol=0.001; total time=   4.8s
[CV] END C=2.0, degree=3, gamma=scale, kernel=poly, tol=0.001; total time=   4.7s
[CV] END C=2.0, degree=3, gamma=scale, kernel=poly, tol=0.001; total time=   4.7s
[CV] END C=2.0, degree=3, gamma=scale, kernel=poly, tol=0.001; total time=   4.6s
[CV] END C=2.0, degree=3, gamma=scale, kernel=poly, tol=0.003; total time=   4.5s
[CV] END C=2.0, degree=3, gamma=scale, kernel=poly, tol=0.003; total time=   4.6s
[CV] END C=2.0, degree=3, gamma=scale, kernel=poly, tol=0.003; total time=   4.4s
[CV] END C=2.0, degree=3, gamma=scale, kernel=poly, tol=0.003; total time=   4.8s
[CV] END C=2.0, degree=3, gamma=scale, kernel=poly, tol=0.003; total time=   4.9s
[CV] END C=2.0, degree=3, gamma=scale, kernel=rbf, tol=0.001; total time=   5.6s
[CV] END C=2.0,

[CV] END C=2.5, degree=2, gamma=auto, kernel=poly, tol=0.003; total time=   4.1s
[CV] END .C=2.5, degree=2, gamma=auto, kernel=rbf, tol=0.001; total time=   5.1s
[CV] END .C=2.5, degree=2, gamma=auto, kernel=rbf, tol=0.001; total time=   5.2s
[CV] END .C=2.5, degree=2, gamma=auto, kernel=rbf, tol=0.001; total time=   5.1s
[CV] END .C=2.5, degree=2, gamma=auto, kernel=rbf, tol=0.001; total time=   5.2s
[CV] END .C=2.5, degree=2, gamma=auto, kernel=rbf, tol=0.001; total time=   5.2s
[CV] END .C=2.5, degree=2, gamma=auto, kernel=rbf, tol=0.003; total time=   5.0s
[CV] END .C=2.5, degree=2, gamma=auto, kernel=rbf, tol=0.003; total time=   5.3s
[CV] END .C=2.5, degree=2, gamma=auto, kernel=rbf, tol=0.003; total time=   5.0s
[CV] END .C=2.5, degree=2, gamma=auto, kernel=rbf, tol=0.003; total time=   5.1s
[CV] END .C=2.5, degree=2, gamma=auto, kernel=rbf, tol=0.003; total time=   5.1s
[CV] END C=2.5, degree=2, gamma=auto, kernel=sigmoid, tol=0.001; total time=   3.1s
[CV] END C=2.5, degree=2,

GridSearchCV(cv=5, error_score='raise',
             estimator=SVC(class_weight='balanced', probability=True),
             param_grid={'C': [1.0, 1.5, 2.0, 2.5], 'degree': [2, 3],
                         'gamma': ['scale', 'auto'],
                         'kernel': ['poly', 'rbf', 'sigmoid'],
                         'tol': [0.001, 0.003]},
             refit='recall', scoring=['accuracy', 'f1', 'precision', 'recall'],
             verbose=2)

In [12]:
fine_tuned_models.append(svc_clf.best_estimator_)
svc_clf.best_params_

{'C': 2.5, 'degree': 2, 'gamma': 'scale', 'kernel': 'rbf', 'tol': 0.001}

### Extra Trees Classifier

In [13]:
%%time

'''ExtraTreesClassifier
parameters: 
    n_estimators: number of trees in the forest; (default = 100)
                    generally random forests do not overfit as more trees are added, however, the increase on
                    performance does not necessarily justifies the increase on complexity (more time/resources)
                    
    max_depth: maximum depth of each tree in the forest; (default = None, all leaves are expanded as much as possible)
                    generally, the deeper the tree, the more splits, which can lead to higher performance, as it
                    allows the model to better fit the data (lower bias), however this can also lead to higher
                    variance as the model can start to overfit the training data
    
    min_samples_split: minimum number of samples required to split an internal node; (default = 2)
                    this can reduce the number of splits, which can reduce overfitting, however, if too large, it
                    can also lead to underfitting the training set
    
    min_samples_leaf: mininum number of samples that must be present in a leaf/terminal node (end of the tree); (default = 1)
                    as before, a higher value can reduce overfitting but a too high value can lead to underfitting.
                    Specially in regression, a higher value can also smooth the model, by avoiding leaves with only one value
                    
    max_features: number of features to consider for best split (deafault = 'auto', total number of features)
                    a higher number will lead to better performance, but a too high number will also lead to overfitting.
                    Moreover, the higher the number, the more computationally expensive the model becomes.
    
    max_samples: number (or %) of samples to draw from X to train each base estimator (default = None, total num of samples)
                     The lower the value, the more randomness is introduced to the data.
                     --Only possible if Bootstrap = True.
    '''

param_grid = {'n_estimators':[ 500], #200, 250,
              'max_depth':[28, 29, 30],
              'min_samples_split':[2, 3, 4],
              'min_samples_leaf':[1, 2],
              'max_features':[4, 5, 6, 7],
              'max_samples':[0.9, 1.0]
             }

et_clf = GridSearchCV(estimator = ExtraTreesClassifier(random_state = 42, n_jobs = -1, bootstrap  = True),
                      param_grid = param_grid, 
                      cv = 10, 
                      scoring = metrics,
                      refit = 'recall',
                      error_score = 'raise', 
                      verbose = 2)
                     
et_clf.fit(X_train, y_train)

Fitting 10 folds for each of 144 candidates, totalling 1440 fits
[CV] END max_depth=28, max_features=4, max_samples=0.9, min_samples_leaf=1, min_samples_split=2, n_estimators=500; total time=   2.7s
[CV] END max_depth=28, max_features=4, max_samples=0.9, min_samples_leaf=1, min_samples_split=2, n_estimators=500; total time=   0.9s
[CV] END max_depth=28, max_features=4, max_samples=0.9, min_samples_leaf=1, min_samples_split=2, n_estimators=500; total time=   0.9s
[CV] END max_depth=28, max_features=4, max_samples=0.9, min_samples_leaf=1, min_samples_split=2, n_estimators=500; total time=   0.9s
[CV] END max_depth=28, max_features=4, max_samples=0.9, min_samples_leaf=1, min_samples_split=2, n_estimators=500; total time=   1.0s
[CV] END max_depth=28, max_features=4, max_samples=0.9, min_samples_leaf=1, min_samples_split=2, n_estimators=500; total time=   0.9s
[CV] END max_depth=28, max_features=4, max_samples=0.9, min_samples_leaf=1, min_samples_split=2, n_estimators=500; total time=   0.

[CV] END max_depth=28, max_features=4, max_samples=1.0, min_samples_leaf=1, min_samples_split=2, n_estimators=500; total time=   1.0s
[CV] END max_depth=28, max_features=4, max_samples=1.0, min_samples_leaf=1, min_samples_split=2, n_estimators=500; total time=   1.0s
[CV] END max_depth=28, max_features=4, max_samples=1.0, min_samples_leaf=1, min_samples_split=2, n_estimators=500; total time=   1.0s
[CV] END max_depth=28, max_features=4, max_samples=1.0, min_samples_leaf=1, min_samples_split=2, n_estimators=500; total time=   0.9s
[CV] END max_depth=28, max_features=4, max_samples=1.0, min_samples_leaf=1, min_samples_split=2, n_estimators=500; total time=   1.0s
[CV] END max_depth=28, max_features=4, max_samples=1.0, min_samples_leaf=1, min_samples_split=2, n_estimators=500; total time=   0.9s
[CV] END max_depth=28, max_features=4, max_samples=1.0, min_samples_leaf=1, min_samples_split=2, n_estimators=500; total time=   0.9s
[CV] END max_depth=28, max_features=4, max_samples=1.0, min_sa

[CV] END max_depth=28, max_features=5, max_samples=0.9, min_samples_leaf=1, min_samples_split=2, n_estimators=500; total time=   0.9s
[CV] END max_depth=28, max_features=5, max_samples=0.9, min_samples_leaf=1, min_samples_split=2, n_estimators=500; total time=   0.9s
[CV] END max_depth=28, max_features=5, max_samples=0.9, min_samples_leaf=1, min_samples_split=2, n_estimators=500; total time=   0.9s
[CV] END max_depth=28, max_features=5, max_samples=0.9, min_samples_leaf=1, min_samples_split=2, n_estimators=500; total time=   1.0s
[CV] END max_depth=28, max_features=5, max_samples=0.9, min_samples_leaf=1, min_samples_split=2, n_estimators=500; total time=   0.9s
[CV] END max_depth=28, max_features=5, max_samples=0.9, min_samples_leaf=1, min_samples_split=2, n_estimators=500; total time=   1.0s
[CV] END max_depth=28, max_features=5, max_samples=0.9, min_samples_leaf=1, min_samples_split=2, n_estimators=500; total time=   0.9s
[CV] END max_depth=28, max_features=5, max_samples=0.9, min_sa

[CV] END max_depth=28, max_features=5, max_samples=1.0, min_samples_leaf=1, min_samples_split=2, n_estimators=500; total time=   0.9s
[CV] END max_depth=28, max_features=5, max_samples=1.0, min_samples_leaf=1, min_samples_split=2, n_estimators=500; total time=   0.9s
[CV] END max_depth=28, max_features=5, max_samples=1.0, min_samples_leaf=1, min_samples_split=2, n_estimators=500; total time=   0.9s
[CV] END max_depth=28, max_features=5, max_samples=1.0, min_samples_leaf=1, min_samples_split=2, n_estimators=500; total time=   0.9s
[CV] END max_depth=28, max_features=5, max_samples=1.0, min_samples_leaf=1, min_samples_split=2, n_estimators=500; total time=   1.0s
[CV] END max_depth=28, max_features=5, max_samples=1.0, min_samples_leaf=1, min_samples_split=3, n_estimators=500; total time=   1.0s
[CV] END max_depth=28, max_features=5, max_samples=1.0, min_samples_leaf=1, min_samples_split=3, n_estimators=500; total time=   1.0s
[CV] END max_depth=28, max_features=5, max_samples=1.0, min_sa

[CV] END max_depth=28, max_features=6, max_samples=0.9, min_samples_leaf=1, min_samples_split=2, n_estimators=500; total time=   0.9s
[CV] END max_depth=28, max_features=6, max_samples=0.9, min_samples_leaf=1, min_samples_split=2, n_estimators=500; total time=   0.8s
[CV] END max_depth=28, max_features=6, max_samples=0.9, min_samples_leaf=1, min_samples_split=2, n_estimators=500; total time=   0.9s
[CV] END max_depth=28, max_features=6, max_samples=0.9, min_samples_leaf=1, min_samples_split=3, n_estimators=500; total time=   0.8s
[CV] END max_depth=28, max_features=6, max_samples=0.9, min_samples_leaf=1, min_samples_split=3, n_estimators=500; total time=   0.8s
[CV] END max_depth=28, max_features=6, max_samples=0.9, min_samples_leaf=1, min_samples_split=3, n_estimators=500; total time=   0.9s
[CV] END max_depth=28, max_features=6, max_samples=0.9, min_samples_leaf=1, min_samples_split=3, n_estimators=500; total time=   0.9s
[CV] END max_depth=28, max_features=6, max_samples=0.9, min_sa

...

[CV] END max_depth=30, max_features=6, max_samples=0.9, min_samples_leaf=2, min_samples_split=2, n_estimators=500; total time=   0.9s
[CV] END max_depth=30, max_features=6, max_samples=0.9, min_samples_leaf=2, min_samples_split=3, n_estimators=500; total time=   0.9s
[CV] END max_depth=30, max_features=6, max_samples=0.9, min_samples_leaf=2, min_samples_split=3, n_estimators=500; total time=   0.9s
[CV] END max_depth=30, max_features=6, max_samples=0.9, min_samples_leaf=2, min_samples_split=3, n_estimators=500; total time=   1.0s
[CV] END max_depth=30, max_features=6, max_samples=0.9, min_samples_leaf=2, min_samples_split=3, n_estimators=500; total time=   1.0s
[CV] END max_depth=30, max_features=6, max_samples=0.9, min_samples_leaf=2, min_samples_split=3, n_estimators=500; total time=   1.0s
[CV] END max_depth=30, max_features=6, max_samples=0.9, min_samples_leaf=2, min_samples_split=3, n_estimators=500; total time=   0.9s
[CV] END max_depth=30, max_features=6, max_samples=0.9, min_sa

[CV] END max_depth=30, max_features=6, max_samples=1.0, min_samples_leaf=2, min_samples_split=3, n_estimators=500; total time=   1.0s
[CV] END max_depth=30, max_features=6, max_samples=1.0, min_samples_leaf=2, min_samples_split=3, n_estimators=500; total time=   0.9s
[CV] END max_depth=30, max_features=6, max_samples=1.0, min_samples_leaf=2, min_samples_split=3, n_estimators=500; total time=   0.9s
[CV] END max_depth=30, max_features=6, max_samples=1.0, min_samples_leaf=2, min_samples_split=3, n_estimators=500; total time=   0.9s
[CV] END max_depth=30, max_features=6, max_samples=1.0, min_samples_leaf=2, min_samples_split=3, n_estimators=500; total time=   0.9s
[CV] END max_depth=30, max_features=6, max_samples=1.0, min_samples_leaf=2, min_samples_split=3, n_estimators=500; total time=   0.9s
[CV] END max_depth=30, max_features=6, max_samples=1.0, min_samples_leaf=2, min_samples_split=3, n_estimators=500; total time=   0.9s
[CV] END max_depth=30, max_features=6, max_samples=1.0, min_sa

[CV] END max_depth=30, max_features=7, max_samples=0.9, min_samples_leaf=2, min_samples_split=3, n_estimators=500; total time=   1.0s
[CV] END max_depth=30, max_features=7, max_samples=0.9, min_samples_leaf=2, min_samples_split=3, n_estimators=500; total time=   1.0s
[CV] END max_depth=30, max_features=7, max_samples=0.9, min_samples_leaf=2, min_samples_split=3, n_estimators=500; total time=   1.0s
[CV] END max_depth=30, max_features=7, max_samples=0.9, min_samples_leaf=2, min_samples_split=3, n_estimators=500; total time=   0.9s
[CV] END max_depth=30, max_features=7, max_samples=0.9, min_samples_leaf=2, min_samples_split=3, n_estimators=500; total time=   0.9s
[CV] END max_depth=30, max_features=7, max_samples=0.9, min_samples_leaf=2, min_samples_split=3, n_estimators=500; total time=   1.0s
[CV] END max_depth=30, max_features=7, max_samples=0.9, min_samples_leaf=2, min_samples_split=3, n_estimators=500; total time=   1.0s
[CV] END max_depth=30, max_features=7, max_samples=0.9, min_sa

[CV] END max_depth=30, max_features=7, max_samples=1.0, min_samples_leaf=2, min_samples_split=3, n_estimators=500; total time=   1.0s
[CV] END max_depth=30, max_features=7, max_samples=1.0, min_samples_leaf=2, min_samples_split=3, n_estimators=500; total time=   1.0s
[CV] END max_depth=30, max_features=7, max_samples=1.0, min_samples_leaf=2, min_samples_split=3, n_estimators=500; total time=   1.0s
[CV] END max_depth=30, max_features=7, max_samples=1.0, min_samples_leaf=2, min_samples_split=3, n_estimators=500; total time=   0.9s
[CV] END max_depth=30, max_features=7, max_samples=1.0, min_samples_leaf=2, min_samples_split=3, n_estimators=500; total time=   0.9s
[CV] END max_depth=30, max_features=7, max_samples=1.0, min_samples_leaf=2, min_samples_split=4, n_estimators=500; total time=   0.9s
[CV] END max_depth=30, max_features=7, max_samples=1.0, min_samples_leaf=2, min_samples_split=4, n_estimators=500; total time=   1.0s
[CV] END max_depth=30, max_features=7, max_samples=1.0, min_sa

GridSearchCV(cv=10, error_score='raise',
             estimator=ExtraTreesClassifier(bootstrap=True, n_jobs=-1,
                                            random_state=42),
             param_grid={'max_depth': [28, 29, 30],
                         'max_features': [4, 5, 6, 7],
                         'max_samples': [0.9, 1.0], 'min_samples_leaf': [1, 2],
                         'min_samples_split': [2, 3, 4],
                         'n_estimators': [500]},
             refit='recall', scoring=['accuracy', 'f1', 'precision', 'recall'],
             verbose=2)

In [14]:
fine_tuned_models.append(et_clf.best_estimator_)
et_clf.best_params_

{'max_depth': 29,
 'max_features': 6,
 'max_samples': 1.0,
 'min_samples_leaf': 1,
 'min_samples_split': 3,
 'n_estimators': 500}

### XGBClassifier

In [15]:
%%time
# https://xgboost.readthedocs.io/en/stable/python/python_api.html?highlight=xgboost%20xgbregressor#xgboost.XGBRegressor
'''XGBClassifier
parameters: 
    n_estimators: number of gradient boosted trees;
                    
    max_depth: maximum depth of each tree; (default = 6)
                    a higher value leads to a more complex model, more likely to overfit.
    
    learning_rate: boosting learning rate; (default = 0.3)
                    it shrinks the feature weights to make the boosting process more conservative. It prevents overfitting.
    
    subsample: subsample ratio if the training instances; (default = 1)
                    a lower value can help prevent overfitting.
   
    colsample_bytree: subsample ratio of columns (features) when constructing each tree; (default = 1)
                    similar to max_features in RandomForest/ExtraTrees
                    
    reg_alpha: L1 regularization term on weights; (default = 1)
                    increasing this value can prevent overfitting
                    
    reg_lambda: L2 regularization term on weights; (default = 0)
                    increasing this value can prevent overfitting
    '''

param_grid = {'n_estimators':[500],
              'max_depth':[6, 14, 18],
              'learning_rate':[0.05, 0.1, 0.15],
              'subsample':[0.9, 1.0],
              'colsample_bytree':[0.8, 0.9, 1.],
              'reg_lambda':[1.0, 1.2, 1.4],
              'n_jobs':[-1],
              'random_state':[42]}

xgb_clf = GridSearchCV(estimator = XGBClassifier(use_label_encoder=False),
                      param_grid = param_grid, 
                      cv = 10, 
                      scoring = metrics,
                      refit = 'recall',
                      error_score = 'raise', 
                      verbose = 2)
                     
xgb_clf.fit(X_train, y_train, eval_metric = 'logloss')

Fitting 10 folds for each of 162 candidates, totalling 1620 fits
[CV] END colsample_bytree=0.8, learning_rate=0.05, max_depth=6, n_estimators=500, n_jobs=-1, random_state=42, reg_lambda=1.0, subsample=0.9; total time=   2.0s
[CV] END colsample_bytree=0.8, learning_rate=0.05, max_depth=6, n_estimators=500, n_jobs=-1, random_state=42, reg_lambda=1.0, subsample=0.9; total time=   1.9s
[CV] END colsample_bytree=0.8, learning_rate=0.05, max_depth=6, n_estimators=500, n_jobs=-1, random_state=42, reg_lambda=1.0, subsample=0.9; total time=   1.9s
[CV] END colsample_bytree=0.8, learning_rate=0.05, max_depth=6, n_estimators=500, n_jobs=-1, random_state=42, reg_lambda=1.0, subsample=0.9; total time=   1.9s
[CV] END colsample_bytree=0.8, learning_rate=0.05, max_depth=6, n_estimators=500, n_jobs=-1, random_state=42, reg_lambda=1.0, subsample=0.9; total time=   1.9s
[CV] END colsample_bytree=0.8, learning_rate=0.05, max_depth=6, n_estimators=500, n_jobs=-1, random_state=42, reg_lambda=1.0, subsample

[CV] END colsample_bytree=0.8, learning_rate=0.05, max_depth=6, n_estimators=500, n_jobs=-1, random_state=42, reg_lambda=1.4, subsample=1.0; total time=   1.9s
[CV] END colsample_bytree=0.8, learning_rate=0.05, max_depth=6, n_estimators=500, n_jobs=-1, random_state=42, reg_lambda=1.4, subsample=1.0; total time=   1.9s
[CV] END colsample_bytree=0.8, learning_rate=0.05, max_depth=6, n_estimators=500, n_jobs=-1, random_state=42, reg_lambda=1.4, subsample=1.0; total time=   2.1s
[CV] END colsample_bytree=0.8, learning_rate=0.05, max_depth=6, n_estimators=500, n_jobs=-1, random_state=42, reg_lambda=1.4, subsample=1.0; total time=   2.1s
[CV] END colsample_bytree=0.8, learning_rate=0.05, max_depth=6, n_estimators=500, n_jobs=-1, random_state=42, reg_lambda=1.4, subsample=1.0; total time=   1.9s
[CV] END colsample_bytree=0.8, learning_rate=0.05, max_depth=6, n_estimators=500, n_jobs=-1, random_state=42, reg_lambda=1.4, subsample=1.0; total time=   1.9s
[CV] END colsample_bytree=0.8, learning_

[CV] END colsample_bytree=0.8, learning_rate=0.05, max_depth=14, n_estimators=500, n_jobs=-1, random_state=42, reg_lambda=1.4, subsample=0.9; total time=   3.3s
[CV] END colsample_bytree=0.8, learning_rate=0.05, max_depth=14, n_estimators=500, n_jobs=-1, random_state=42, reg_lambda=1.4, subsample=0.9; total time=   3.3s
[CV] END colsample_bytree=0.8, learning_rate=0.05, max_depth=14, n_estimators=500, n_jobs=-1, random_state=42, reg_lambda=1.4, subsample=0.9; total time=   3.1s
[CV] END colsample_bytree=0.8, learning_rate=0.05, max_depth=14, n_estimators=500, n_jobs=-1, random_state=42, reg_lambda=1.4, subsample=0.9; total time=   3.2s
[CV] END colsample_bytree=0.8, learning_rate=0.05, max_depth=14, n_estimators=500, n_jobs=-1, random_state=42, reg_lambda=1.4, subsample=0.9; total time=   3.4s
[CV] END colsample_bytree=0.8, learning_rate=0.05, max_depth=14, n_estimators=500, n_jobs=-1, random_state=42, reg_lambda=1.4, subsample=0.9; total time=   3.5s
[CV] END colsample_bytree=0.8, lea

[CV] END colsample_bytree=0.8, learning_rate=0.05, max_depth=18, n_estimators=500, n_jobs=-1, random_state=42, reg_lambda=1.2, subsample=1.0; total time=   2.9s
[CV] END colsample_bytree=0.8, learning_rate=0.05, max_depth=18, n_estimators=500, n_jobs=-1, random_state=42, reg_lambda=1.2, subsample=1.0; total time=   2.8s
[CV] END colsample_bytree=0.8, learning_rate=0.05, max_depth=18, n_estimators=500, n_jobs=-1, random_state=42, reg_lambda=1.2, subsample=1.0; total time=   3.0s
[CV] END colsample_bytree=0.8, learning_rate=0.05, max_depth=18, n_estimators=500, n_jobs=-1, random_state=42, reg_lambda=1.2, subsample=1.0; total time=   3.1s
[CV] END colsample_bytree=0.8, learning_rate=0.05, max_depth=18, n_estimators=500, n_jobs=-1, random_state=42, reg_lambda=1.2, subsample=1.0; total time=   2.9s
[CV] END colsample_bytree=0.8, learning_rate=0.05, max_depth=18, n_estimators=500, n_jobs=-1, random_state=42, reg_lambda=1.2, subsample=1.0; total time=   2.9s
[CV] END colsample_bytree=0.8, lea

[CV] END colsample_bytree=0.8, learning_rate=0.1, max_depth=6, n_estimators=500, n_jobs=-1, random_state=42, reg_lambda=1.2, subsample=0.9; total time=   3.4s
[CV] END colsample_bytree=0.8, learning_rate=0.1, max_depth=6, n_estimators=500, n_jobs=-1, random_state=42, reg_lambda=1.2, subsample=0.9; total time=   3.5s
[CV] END colsample_bytree=0.8, learning_rate=0.1, max_depth=6, n_estimators=500, n_jobs=-1, random_state=42, reg_lambda=1.2, subsample=0.9; total time=   3.7s
[CV] END colsample_bytree=0.8, learning_rate=0.1, max_depth=6, n_estimators=500, n_jobs=-1, random_state=42, reg_lambda=1.2, subsample=0.9; total time=   3.3s
[CV] END colsample_bytree=0.8, learning_rate=0.1, max_depth=6, n_estimators=500, n_jobs=-1, random_state=42, reg_lambda=1.2, subsample=0.9; total time=   3.3s
[CV] END colsample_bytree=0.8, learning_rate=0.1, max_depth=6, n_estimators=500, n_jobs=-1, random_state=42, reg_lambda=1.2, subsample=1.0; total time=   3.0s
[CV] END colsample_bytree=0.8, learning_rate=0

...

[CV] END colsample_bytree=1.0, learning_rate=0.15, max_depth=6, n_estimators=500, n_jobs=-1, random_state=42, reg_lambda=1.0, subsample=0.9; total time=   2.9s
[CV] END colsample_bytree=1.0, learning_rate=0.15, max_depth=6, n_estimators=500, n_jobs=-1, random_state=42, reg_lambda=1.0, subsample=0.9; total time=   2.9s
[CV] END colsample_bytree=1.0, learning_rate=0.15, max_depth=6, n_estimators=500, n_jobs=-1, random_state=42, reg_lambda=1.0, subsample=0.9; total time=   2.7s
[CV] END colsample_bytree=1.0, learning_rate=0.15, max_depth=6, n_estimators=500, n_jobs=-1, random_state=42, reg_lambda=1.0, subsample=0.9; total time=   2.7s
[CV] END colsample_bytree=1.0, learning_rate=0.15, max_depth=6, n_estimators=500, n_jobs=-1, random_state=42, reg_lambda=1.0, subsample=0.9; total time=   2.9s
[CV] END colsample_bytree=1.0, learning_rate=0.15, max_depth=6, n_estimators=500, n_jobs=-1, random_state=42, reg_lambda=1.0, subsample=1.0; total time=   2.4s
[CV] END colsample_bytree=1.0, learning_

[CV] END colsample_bytree=1.0, learning_rate=0.15, max_depth=6, n_estimators=500, n_jobs=-1, random_state=42, reg_lambda=1.4, subsample=1.0; total time=   2.6s
[CV] END colsample_bytree=1.0, learning_rate=0.15, max_depth=6, n_estimators=500, n_jobs=-1, random_state=42, reg_lambda=1.4, subsample=1.0; total time=   2.7s
[CV] END colsample_bytree=1.0, learning_rate=0.15, max_depth=6, n_estimators=500, n_jobs=-1, random_state=42, reg_lambda=1.4, subsample=1.0; total time=   2.8s
[CV] END colsample_bytree=1.0, learning_rate=0.15, max_depth=14, n_estimators=500, n_jobs=-1, random_state=42, reg_lambda=1.0, subsample=0.9; total time=   3.0s
[CV] END colsample_bytree=1.0, learning_rate=0.15, max_depth=14, n_estimators=500, n_jobs=-1, random_state=42, reg_lambda=1.0, subsample=0.9; total time=   3.2s
[CV] END colsample_bytree=1.0, learning_rate=0.15, max_depth=14, n_estimators=500, n_jobs=-1, random_state=42, reg_lambda=1.0, subsample=0.9; total time=   3.4s
[CV] END colsample_bytree=1.0, learni

[CV] END colsample_bytree=1.0, learning_rate=0.15, max_depth=14, n_estimators=500, n_jobs=-1, random_state=42, reg_lambda=1.4, subsample=0.9; total time=   3.3s
[CV] END colsample_bytree=1.0, learning_rate=0.15, max_depth=14, n_estimators=500, n_jobs=-1, random_state=42, reg_lambda=1.4, subsample=0.9; total time=   3.4s
[CV] END colsample_bytree=1.0, learning_rate=0.15, max_depth=14, n_estimators=500, n_jobs=-1, random_state=42, reg_lambda=1.4, subsample=1.0; total time=   3.3s
[CV] END colsample_bytree=1.0, learning_rate=0.15, max_depth=14, n_estimators=500, n_jobs=-1, random_state=42, reg_lambda=1.4, subsample=1.0; total time=   3.3s
[CV] END colsample_bytree=1.0, learning_rate=0.15, max_depth=14, n_estimators=500, n_jobs=-1, random_state=42, reg_lambda=1.4, subsample=1.0; total time=   3.5s
[CV] END colsample_bytree=1.0, learning_rate=0.15, max_depth=14, n_estimators=500, n_jobs=-1, random_state=42, reg_lambda=1.4, subsample=1.0; total time=   3.2s
[CV] END colsample_bytree=1.0, lea

[CV] END colsample_bytree=1.0, learning_rate=0.15, max_depth=18, n_estimators=500, n_jobs=-1, random_state=42, reg_lambda=1.2, subsample=1.0; total time=   3.2s
[CV] END colsample_bytree=1.0, learning_rate=0.15, max_depth=18, n_estimators=500, n_jobs=-1, random_state=42, reg_lambda=1.4, subsample=0.9; total time=   3.2s
[CV] END colsample_bytree=1.0, learning_rate=0.15, max_depth=18, n_estimators=500, n_jobs=-1, random_state=42, reg_lambda=1.4, subsample=0.9; total time=   3.3s
[CV] END colsample_bytree=1.0, learning_rate=0.15, max_depth=18, n_estimators=500, n_jobs=-1, random_state=42, reg_lambda=1.4, subsample=0.9; total time=   3.4s
[CV] END colsample_bytree=1.0, learning_rate=0.15, max_depth=18, n_estimators=500, n_jobs=-1, random_state=42, reg_lambda=1.4, subsample=0.9; total time=   3.4s
[CV] END colsample_bytree=1.0, learning_rate=0.15, max_depth=18, n_estimators=500, n_jobs=-1, random_state=42, reg_lambda=1.4, subsample=0.9; total time=   3.6s
[CV] END colsample_bytree=1.0, lea

GridSearchCV(cv=10, error_score='raise',
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     enable_categorical=False, gamma=None,
                                     gpu_id=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monot...
                                     subsample=None, tree_method=None,
                                     use_label_encoder=False,
                                     validate_parameters=None, verbosity=None),
             param_grid={'colsample_bytree': [0.8, 0.9, 1.0],
        

In [16]:
fine_tuned_models.append(xgb_clf.best_estimator_)
xgb_clf.best_params_

{'colsample_bytree': 1.0,
 'learning_rate': 0.15,
 'max_depth': 18,
 'n_estimators': 500,
 'n_jobs': -1,
 'random_state': 42,
 'reg_lambda': 1.4,
 'subsample': 0.9}

## Comparing models

In [17]:
%%time
for model in fine_tuned_models:
    cross_val = cross_validate(model, X_train, y_train, cv= 10, scoring = metrics)

    new_models.append(f'{model.__class__.__name__}')
    new_scores.append(list(cross_val.values())[2:])

data = []
for model in range(len(fine_tuned_models)):
    for score in range(len(metrics)):
        data.append(new_scores[model][score].mean())

reshaped_data = np.reshape(data, (len(fine_tuned_models), len(metrics)))
        
df_models = pd.DataFrame(reshaped_data, index = new_models, columns = metrics)
df_models.style.highlight_max(color = 'lightgreen', axis = 0)

Wall time: 2min 18s


## Combining models

### VotingClassifier

In [18]:
%%time
'''VotingClassifier
parameters: 
    weights: sequence of weights (float or int) to weight the occurrences of predicted class labels (hard voting) 
            or class probabilities before averaging (soft voting).

    '''

param_grid = {'weights':[[0.25, 0.25, 0.25, 0.25],
                         [0.15, 0.15, 0.4, 0.3],
                         [0.2, 0.1, 0.4, 0.3],
                         [0.2, 0.1, 0.45, 0.25],
                         [0.25, 0.05, 0.4, 0.3],
                         [0.25, 0.05, 0.45, 0.25],
                         [0.3, 0.05, 0.45, 0.2],
                         [0.15, 0.15, 0.45, 0.25],
                         [0.2, 0.1, 0.3, 0.4]]}

vot_clf = GridSearchCV(estimator = VotingClassifier(estimators=[('knn', knn_clf.best_estimator_),
                                                                     ('svc', svc_clf.best_estimator_),
                                                                     ('extra_trees', et_clf.best_estimator_), 
                                                                     ('xgboosting', xgb_clf.best_estimator_)],
                                                      voting = 'soft',
                                                      n_jobs = -1),
                      param_grid = param_grid, 
                      cv = 10, 
                      scoring = metrics,
                      refit = 'recall',
                      error_score = 'raise', 
                      verbose = 2)
                     
vot_clf.fit(X_train, y_train)
vot_clf.best_params_

Fitting 10 folds for each of 9 candidates, totalling 90 fits
[CV] END ...................weights=[0.25, 0.25, 0.25, 0.25]; total time=   9.3s
[CV] END ...................weights=[0.25, 0.25, 0.25, 0.25]; total time=   9.2s
[CV] END ...................weights=[0.25, 0.25, 0.25, 0.25]; total time=   9.7s
[CV] END ...................weights=[0.25, 0.25, 0.25, 0.25]; total time=   9.5s
[CV] END ...................weights=[0.25, 0.25, 0.25, 0.25]; total time=   9.2s
[CV] END ...................weights=[0.25, 0.25, 0.25, 0.25]; total time=   9.7s
[CV] END ...................weights=[0.25, 0.25, 0.25, 0.25]; total time=   9.6s
[CV] END ...................weights=[0.25, 0.25, 0.25, 0.25]; total time=   9.3s
[CV] END ...................weights=[0.25, 0.25, 0.25, 0.25]; total time=   9.7s
[CV] END ...................weights=[0.25, 0.25, 0.25, 0.25]; total time=   9.5s
[CV] END .....................weights=[0.15, 0.15, 0.4, 0.3]; total time=   9.2s
[CV] END .....................weights=[0.15, 0.1

{'weights': [0.3, 0.05, 0.45, 0.2]}

In [19]:
fine_tuned_models.append(vot_clf.best_estimator_)

In [20]:
%%time
cross_val = cross_validate(vot_clf.best_estimator_, X_train, y_train, cv= 10, scoring = metrics)
new_models.append(f'{vot_clf.best_estimator_.__class__.__name__}')
new_scores.append(list(cross_val.values())[2:])
        

for score in range(len(metrics)):
    data.append(new_scores[-1][score].mean())

reshaped_data = np.reshape(data, (len(fine_tuned_models), len(metrics)))
        
df_models = pd.DataFrame(reshaped_data, index = new_models, columns = metrics)
df_models.style.highlight_max(color = 'lightgreen', axis = 0)

Wall time: 1min 37s


## Stacking Classifier

In [21]:
%%time
'''StackingClassifier
parameters: 
    weights: sequence of weights (float or int) to weight the occurrences of predicted class labels (hard voting) 
            or class probabilities before averaging (soft voting).

    '''

param_grid = {'final_estimator':exp_models,
              'passthrough':[True, False]}

stk_clf = GridSearchCV(estimator = StackingClassifier(estimators=[('knn', knn_clf.best_estimator_),
                                                                     ('svc', svc_clf.best_estimator_),
                                                                     ('extra_trees', et_clf.best_estimator_), 
                                                                     ('xgboosting', xgb_clf.best_estimator_)],
                                                      n_jobs = -1),
                      param_grid = param_grid, 
                      cv = 10, 
                      scoring = metrics,
                      refit = 'recall',
                      error_score = 'raise', 
                      verbose = 2)
                     
stk_clf.fit(X_train, y_train)
stk_clf.best_params_

Fitting 10 folds for each of 18 candidates, totalling 180 fits
[CV] END final_estimator=LogisticRegression(), passthrough=True; total time=  22.6s
[CV] END final_estimator=LogisticRegression(), passthrough=True; total time=  27.8s
[CV] END final_estimator=LogisticRegression(), passthrough=True; total time=  28.6s
[CV] END final_estimator=LogisticRegression(), passthrough=True; total time=  24.7s
[CV] END final_estimator=LogisticRegression(), passthrough=True; total time=  32.4s
[CV] END final_estimator=LogisticRegression(), passthrough=True; total time=  27.4s
[CV] END final_estimator=LogisticRegression(), passthrough=True; total time=  23.6s
[CV] END final_estimator=LogisticRegression(), passthrough=True; total time=  27.4s
[CV] END final_estimator=LogisticRegression(), passthrough=True; total time=  27.8s
[CV] END final_estimator=LogisticRegression(), passthrough=True; total time=  33.8s
[CV] END final_estimator=LogisticRegression(), passthrough=False; total time=  27.9s
[CV] END fin

[CV] END final_estimator=RandomForestClassifier(random_state=42), passthrough=False; total time=  31.5s
[CV] END final_estimator=RandomForestClassifier(random_state=42), passthrough=False; total time=  30.8s
[CV] END final_estimator=RandomForestClassifier(random_state=42), passthrough=False; total time=  23.8s
[CV] END final_estimator=RandomForestClassifier(random_state=42), passthrough=False; total time=  32.0s
[CV] END final_estimator=RandomForestClassifier(random_state=42), passthrough=False; total time=  31.1s
[CV] END final_estimator=RandomForestClassifier(random_state=42), passthrough=False; total time=  25.2s
[CV] END final_estimator=RandomForestClassifier(random_state=42), passthrough=False; total time=  31.2s
[CV] END final_estimator=RandomForestClassifier(random_state=42), passthrough=False; total time=  32.3s
[CV] END final_estimator=RandomForestClassifier(random_state=42), passthrough=False; total time=  22.5s
[CV] END final_estimator=ExtraTreesClassifier(random_state=42), 

[CV] END final_estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=12,
              num_parallel_tree=1, predictor='auto', random_state=42,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', use_label_encoder=False,
              validate_parameters=1, verbosity=None), passthrough=True; total time=  43.7s
[CV] END final_estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_co

[CV] END final_estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=12,
              num_parallel_tree=1, predictor='auto', random_state=42,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', use_label_encoder=False,
              validate_parameters=1, verbosity=None), passthrough=False; total time=  28.3s
[CV] END final_estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_c

{'final_estimator': KNeighborsClassifier(), 'passthrough': True}

In [22]:
fine_tuned_models.append(stk_clf.best_estimator_)

In [23]:
%%time
cross_val = cross_validate(stk_clf.best_estimator_, X_train, y_train, cv= 10, scoring = metrics)
new_models.append(f'{stk_clf.best_estimator_.__class__.__name__}')
new_scores.append(list(cross_val.values())[2:])
        

for score in range(len(metrics)):
    data.append(new_scores[-1][score].mean())

reshaped_data = np.reshape(data, (len(fine_tuned_models), len(metrics)))
        
df_models = pd.DataFrame(reshaped_data, index = new_models, columns = metrics)
df_models.style.highlight_max(color = 'lightgreen', axis = 0)

Wall time: 6min 2s


# Exporting best model

In [24]:
filename = 'vot_clf.pkl'
pickle.dump(vot_clf.best_estimator_, open(filename, 'wb'))

<!-- # saving best estimator
filename = 'vot_clf.pkl'
pickle.dump(vot_clf_gd.best_estimator_, open(filename, 'wb')) -->